In [1]:
import wave
#import pydub
import numpy as np
import pandas as pd
import librosa
import os
import matplotlib.pyplot as plt
#import keras
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
def splitting(name):
    return [os.path.splitext(name)[1]],  os.path.splitext(name)[0].split(';')[:-1]

In [3]:
#Каталог из которого будем брать файлы
directory = './Voices/'
#Получаем список файлов в переменную files
files = os.listdir(directory) 

In [4]:
#cut the middle values and make 10 audios from 1
lst = []
min_len = 110584
constant = min_len // 10
for i in range(len(files)):
    print(files[i])
    if files[i][0]=='1' or files[i][0]=='2':
        y, sr = librosa.load(directory+files[i])
        extension, name = splitting(files[i])
        interval = (len(y) - min_len) // 2
        for j in range(10):
            
            temp = []
            for k in range(constant):
                temp.append(y[interval+constant*j+k])
#                 lst.append(name+extension+[y[interval+constant*j+k]]+[sr])
#                 print(lst)
            lst.append(name + extension + temp + [sr])
        print('{} out of {}'.format(i, len(files)))

2;Brykalova I;23;F;.wav
0 out of 110
1;Christosova N;38;F;.wav
1 out of 110
2;Dymov N;22;M;.wav
2 out of 110
2;Nesterov V;21;M;.wav
3 out of 110
2;Frolova S;20;F;.wav
4 out of 110
2;Zharov A;22;M;.wav
5 out of 110
2;Danilina U;13;F;.wav
6 out of 110
1;Litvinov S;21;M;.wav
7 out of 110
2;Otradnov S;18;M;.wav
8 out of 110
1;Hertek A;21;F;.wav
9 out of 110
2;Shabalina V;37;F;.wav
10 out of 110
2;Israilov D;22;M;.wav
11 out of 110
1;Frolova S;20;F;.wav
12 out of 110
1;Petukhov V;25;M;.wav
13 out of 110
1;Welet A;24;F;.wav
14 out of 110
2;Glushkov G;20;M;.wav
15 out of 110
2;Kaliberda E.A;53;F;.wav
16 out of 110
1;Morozov S;21;M;.wav
17 out of 110
1;Sharipov K;21;M;.wav
18 out of 110
2;Hertek A;21;F;.wav
19 out of 110
2;Mamaeva M;21;F;.wav
20 out of 110
2;Harlamova S;42;F;.wav
21 out of 110
1;Kochkova E;19;F;.wav
22 out of 110
1;Mamaeva M;21;F;.wav
23 out of 110
1;Zharov A;22;M;.wav
24 out of 110
2;Koshelkov I;24;M;.wav
25 out of 110
2;Morozov S;21;M;.wav
26 out of 110
2;Welet A;24;F;.wav
2

In [13]:
audios = ['Audio_{}'.format(i) for i in range(constant)]
col_names = ['Num', 'Name', 'Age', 'Gender', 'Ext'] + audios + ['Sr']
data = pd.DataFrame(lst, columns=col_names)
data.Name = data.Name.astype('category')
data.Name = data.Name.cat.rename_categories([i for i in range(data.Name.nunique())])
data.head()

,Num,Name,Age,Gender,Ext,Audio_0,Audio_1,Audio_2,Audio_3,Audio_4,...,Audio_11049,Audio_11050,Audio_11051,Audio_11052,Audio_11053,Audio_11054,Audio_11055,Audio_11056,Audio_11057,Sr
0,2,3,23,F,.wav,0.037744,0.042963,0.045602,0.048476,0.049683,...,0.000911,0.000557,0.001369,0.001584,0.001286,0.000359,-0.000251,0.001169,-0.000582,22050
1,2,3,23,F,.wav,0.001462,0.002927,0.001584,0.001837,0.003543,...,0.041023,0.039918,0.035112,0.024913,0.015266,0.004326,-0.006412,-0.009296,-0.008877,22050
2,2,3,23,F,.wav,-0.014228,-0.018208,-0.013448,-0.011329,-0.014752,...,-0.001355,-0.001332,-0.001974,-0.003394,-0.003573,-0.004455,-0.005650,-0.006153,-0.008063,22050
3,2,3,23,F,.wav,-0.008819,-0.009442,-0.009797,-0.010523,-0.010372,...,0.006796,-0.002877,0.019957,-0.009826,-0.018474,0.012452,0.015637,0.000487,-0.008682,22050
4,2,3,23,F,.wav,0.003194,-0.001142,0.007503,-0.006924,0.012379,...,-0.000781,-0.001412,-0.000841,-0.001009,-0.001068,0.000122,-0.000027,-0.000311,-0.000219,22050


In [35]:
train_ffts = []

for i in range(data.shape[0]):
    elem = data[audios].iloc[i].values
    d = np.abs(librosa.stft(elem, n_fft=1024))
    train_ffts.append(d)

In [36]:
#shape = (1100,513,44)
target = data.Name
train_ffts = np.array(train_ffts)
train_ffts = train_ffts.reshape(1100,513,44,1)
X_train, X_test, y_train, y_test = train_test_split(train_ffts, target, test_size=0.1, shuffle=True)

In [37]:
import tensorflow as tf
import keras
from keras.layers import Convolution2D, MaxPooling2D, Conv2D
from keras.layers import Dense, Dropout, Activation, Flatten

In [69]:
model = keras.models.Sequential()
input_shape = (513,44,1)
model.add(Conv2D(512, kernel_size=(5,5), input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, kernel_size=(5,5)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(5,5)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten()) # Flattening the 2D arrays for fully connected layers
model.add(Dense(200, activation=tf.nn.relu))
model.add(Dropout(0.2))
model.add(Dense(55,activation=tf.nn.softmax)) 

In [70]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

In [75]:
model.fit(x=X_train,y=y_train, epochs=20, batch_size=90) 

Epoch 1/20
990/990 [==============================] - 729s 737ms/step - loss: 2.7756 - acc: 0.3000
Epoch 2/20
990/990 [==============================] - 648s 655ms/step - loss: 2.0633 - acc: 0.4455
Epoch 3/20
990/990 [==============================] - 640s 646ms/step - loss: 1.6152 - acc: 0.5899
Epoch 4/20
990/990 [==============================] - 644s 651ms/step - loss: 1.1417 - acc: 0.7091
Epoch 5/20
990/990 [==============================] - 638s 644ms/step - loss: 1.0346 - acc: 0.7364
Epoch 6/20
990/990 [==============================] - 644s 650ms/step - loss: 0.7554 - acc: 0.8000
Epoch 7/20
990/990 [==============================] - 644s 650ms/step - loss: 0.5628 - acc: 0.8576
Epoch 8/20
360/990 [=========>....................] - ETA: 6:54 - loss: 0.7739 - acc: 0.8194

KeyboardInterrupt: 

In [76]:
pred = model.predict(X_test)
prediction = []
for string in pred:
    a = np.argmax(string)
    prediction.append(a)
print(prediction)

[38, 4, 52, 30, 28, 22, 29, 21, 5, 36, 9, 37, 46, 31, 31, 4, 33, 51, 23, 46, 11, 11, 42, 2, 22, 11, 36, 30, 41, 23, 43, 15, 2, 31, 32, 49, 6, 11, 51, 17, 29, 45, 3, 32, 36, 5, 4, 40, 26, 22, 9, 48, 20, 44, 8, 40, 29, 51, 34, 24, 14, 22, 19, 33, 5, 22, 39, 6, 45, 38, 3, 43, 2, 32, 11, 52, 43, 7, 20, 31, 30, 22, 22, 3, 28, 27, 19, 11, 36, 10, 12, 5, 43, 11, 8, 32, 36, 50, 31, 46, 38, 24, 11, 20, 23, 9, 31, 52, 40, 51]


In [77]:
#print(y_test)

In [78]:
print(accuracy_score(prediction, y_test))

0.4909090909090909
